In [2]:
import pandas as pd
import numpy as np
import pymysql
import json 
from scipy.spatial import distance
import psycopg2

connection = psycopg2.connect(
host = "postgres-23xi-aurora-rds-cluster.cluster-ro-cbywwdy08bsq.us-east-1.rds.amazonaws.com",
port = "5432",
user = "gryffindor",
password = "password2311",
database='dev')


results = pd.read_sql('SELECT vehicle_id, CAST(lap as INTEGER), CAST(lead_lap as INTEGER), CAST(run_type as INTEGER), CAST(ps as INTEGER) FROM timejoin_results', connection)
results = results[results['run_type'] == 3]
results = results.sort_values(['lead_lap','vehicle_id'])
results = results.drop_duplicates(['vehicle_id','lap'])
results['lap'] = results.lap.apply(lambda x: 0 if x in [0,1] else x)

df = pd.read_sql("""SELECT * FROM (
SELECT vehicle_id,CAST(lap as INTEGER), CAST(timestamp as DOUBLE PRECISION),CAST(lap_fraction as DOUBLE PRECISION),CAST(distance_from_white_line AS DOUBLE PRECISION),CAST(lap_fraction_pos AS DOUBLE PRECISION),CAST(lead_lap as INTEGER),flag_state,CAST(position_x as DOUBLE PRECISION),CAST(position_y as DOUBLE PRECISION), ROW_NUMBER() OVER (PARTITION BY vehicle_id, lap ORDER BY timestamp ASC) AS row_num
FROM timejoin_throttle_position
WHERE (CAST(lap_fraction_pos AS DOUBLE PRECISION)) > 0.950
) AS subquery
WHERE row_num = 1;""", connection)

df = pd.merge(df, results, on = ['vehicle_id','lap'])
df = df.sort_values(['vehicle_id','lead_lap_x'])
df['flag_state'] = df.flag_state.apply(lambda x:1 if x == 'Green' else 2)
df['flag_change'] = df.groupby('vehicle_id').flag_state.diff()
df.flag_change.fillna(-1, inplace = True)
restarts = df[df['flag_change'] == -1]

value_counts = restarts['lead_lap_y'].value_counts()
filtered_values = value_counts[value_counts > 15].index.tolist()
filtered_values.sort()

OperationalError: connection to server at "postgres-23xi-aurora-rds-cluster.cluster-ro-cbywwdy08bsq.us-east-1.rds.amazonaws.com" (44.219.52.84), port 5432 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?


In [44]:
import boto3
import psycopg2
import pandas as pd
import warnings
import time
import datetime
import json
from multiprocessing import Pool
import numpy as np
from scipy.spatial import distance
import matplotlib.pyplot as plt
import pymysql
warnings.filterwarnings("ignore")


def calc_inside_distance(position, white_line):
    closest_index = distance.cdist([position], white_line).argmin()
    closest_point = white_line[closest_index]
    distance_to_closest = distance.euclidean(position, closest_point)
    lap_frac = np.round(closest_index/len(white_line),4)
    return distance_to_closest, closest_index, lap_frac
connection = psycopg2.connect(
host = "postgres-23xi-aurora-rds-instance-1.cbywwdy08bsq.us-east-1.rds.amazonaws.com",
port = "5432",
user = "gryffindor",
password = "password2311",
database='dev')
cursor=connection.cursor()
mydb = pymysql.connect(
    host="twenty-three-eleven-db.cc99z5yvqqn0.us-east-1.rds.amazonaws.com",
    port=3306,
    user="23xi-rw-aws-secret",
    password="_zMdAdtJsJ8hTn4DA9sbEaJ69LJTyQqs"
)
sql = """SELECT datname FROM pg_database"""
print(pd.read_sql(sql, con=connection))

     datname
0  template0
1   rdsadmin
2  template1
3   postgres
4        dev


In [26]:
race_id = 5301
last_loop = pd.read_sql(f'SELECT DISTINCT loop_name FROM core.erdp_line_crossing WHERE race_id = {race_id}', mydb)
last_loop = last_loop[last_loop['loop_name'].str.contains('L')]
last_loop = last_loop.sort_values('loop_name').iloc[len(last_loop)-1].values[0]



In [ ]:

with open(f'../../Brainstorming/boundary_files/Kansas.json') as file:
    track = json.load(file)
white_line_x = track['boundary']['inside']['x']
white_line_y = track['boundary']['inside']['y']
white_line = list(zip(white_line_x,white_line_y))


query  = f"SELECT * FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY vehicle_no, CAST(lap_fraction AS SIGNED) ORDER BY location_timestamp DESC) AS row_num FROM core.erdp_location_23kan2 WHERE (CAST(lap_fraction * 10000 AS SIGNED) % 10000) > 9800 AND last_loop_name = 'L7') AS subquery WHERE row_num = 1 AND run_type = 3 AND race_id = {race_id};"

k = pd.read_sql(query, mydb)

k['flag_change'] = k.groupby('vehicle_no').flag_code.diff()
k['lap'] = k['lap_fraction'].astype(str).str.split('.').str[0].astype(float)
k.flag_change.fillna(-1, inplace = True)
k2 = k[k['flag_change'] == -1]
value_counts = k2['lead_lap'].value_counts()

# Filter values that occur more than 3 times
filtered_values = value_counts[value_counts > 16].index.tolist()
filtered_values.sort()

output_csv = f"restart_{race_id}.csv"

with open(output_csv, "w") as file:
    df_list = []
    start = pd.read_sql('SELECT * FROM core.erdp_location_curr WHERE run_type = 3 AND lap_fraction < 2', mydb)
    start = start[start['lap_fraction'] < 1.1]
    start = start[start['lap_fraction'] > 1]
    start = start[start['lead_lap'] == 0]
    start = start[start['last_loop_name'].isin(['SF',f'{last_loop}'])]
    start = start.sort_values('location_timestamp', ascending = True).groupby('vehicle_no').first().reset_index()
    start['pos'] = start.last_loop_timestamp.rank()
    start = start.sort_values('pos').reset_index()
    start[['vehicle_no','location_timestamp','pos']]
    pos = list(zip(start.vehicle_position_x, start.vehicle_position_y))
    dist_white_line = list()
    for j in pos:
        dist_white_line.append(calc_inside_distance(j, white_line)[0])
    start['dist_wl'] = dist_white_line
    start['lane'] = start.dist_wl.apply(lambda x: 'inside' if x <= np.median(dist_white_line) else 'outside')
    start['restart_pos'] = start.groupby('lane').pos.rank()
    start[['vehicle_no','restart_pos','dist_wl', 'lane', 'lap_fraction','pos','last_loop_name']].sort_values(['restart_pos','lane','pos'])
    query = f'SELECT * FROM core.erdp_result WHERE lead_lap = 2 AND race_id = {race_id} AND run_type = 3'
    ret = pd.read_sql(query, mydb)
    if len(ret) > 0:
        sort_out = ret.sort_values('position')[['vehicle_no', 'position']]
        r = start[['vehicle_no','restart_pos','dist_wl', 'lane']].sort_values(['restart_pos','lane']).reset_index().reset_index()
        r['start_pos'] = r['level_0'] + 1
        r = r[['vehicle_no','restart_pos', 'lane', 'start_pos']]
        r = pd.merge(r, sort_out, on = 'vehicle_no')
        r['gain'] = r['start_pos'] - r['position']
        df1 = r.pivot_table(index='restart_pos', columns='lane', values='vehicle_no', aggfunc='first')
        df2 = r.pivot_table(index='restart_pos', columns='lane', values='position', aggfunc='first')
        df3 = r.pivot_table(index='restart_pos', columns='lane', values='gain', aggfunc='first')
        out = pd.concat([df1,df2, df3], axis = 1)
        out.index.names = ['Lane']
        out.insert(2,'V','')
        out.insert(5,'C','')
        out.columns = pd.MultiIndex.from_tuples([('Car # Inside', ''), ('Car # Outside', ''),  ('', ''), ('Position After Sort', ''),('Position After Sort', ''),('', '') ,('Gain/Loss', ''), ('Gain/Loss', '')])
        df_list.append(r)
        file.write(f"Restart 0\n")
        file.write(f'Lap #: 1\n')
        file.write(f'Sort Lap: 3\n')
        out.to_csv(file, index=False)
        file.write("\n")

    for i in range(1,len(filtered_values)):
        restart = k[k['lap'] == filtered_values[i]-1]
        restart['pos'] = restart.last_loop_timestamp.rank()
        pos = list(zip(restart.vehicle_position_x, restart.vehicle_position_y))
        dist_white_line = list()
        for j in pos:
            dist_white_line.append(calc_inside_distance(j, white_line)[0])
        restart['dist_wl'] = dist_white_line
        restart['lane'] = restart.dist_wl.apply(lambda x: 'inside' if x < np.mean(dist_white_line) else 'outside')
        restart['restart_pos'] = restart.groupby('lane').pos.rank()
        restart[['vehicle_no','restart_pos','dist_wl', 'lane', 'lap_fraction']].sort_values(['restart_pos','lane'])
        query = f'SELECT * FROM core.erdp_result WHERE lead_lap = {filtered_values[i] + 2} AND race_id = {race_id} AND run_type = 3'
        ret = pd.read_sql(query, mydb)
        if len(ret) < 10:
            query = f'SELECT * FROM core.erdp_result WHERE lead_lap = {filtered_values[i] + 1} AND race_id = {race_id} AND run_type = 3'
            ret = pd.read_sql(query, mydb)
        if len(ret) > 10:
            print(filtered_values[i])
            sort_out = ret.sort_values('position')[['vehicle_no', 'position']]
            r = restart[['vehicle_no','restart_pos','dist_wl', 'lane']].sort_values(['restart_pos','lane']).reset_index().reset_index()
            r['start_pos'] = r['level_0'] + 1
            r = r[['vehicle_no','restart_pos', 'lane', 'start_pos']]
            r = pd.merge(r, sort_out, on = 'vehicle_no')
            r['gain'] = r['start_pos'] - r['position']
            df1 = r.pivot_table(index='restart_pos', columns='lane', values='vehicle_no', aggfunc='first')
            df2 = r.pivot_table(index='restart_pos', columns='lane', values='position', aggfunc='first')
            df3 = r.pivot_table(index='restart_pos', columns='lane', values='gain', aggfunc='first')
            out = pd.concat([df1,df2, df3], axis = 1)
            out.index.names = ['Lane']
            out.insert(2,'V','')
            out.insert(5,'C','')
            out.columns = pd.MultiIndex.from_tuples([('Car # Inside', ''), ('Car # Outside', ''),  ('', ''), ('Position After Sort', ''),('Position After Sort', ''),('', '') ,('Gain/Loss', ''), ('Gain/Loss', '')])
            df_list.append(r)
            file.write(f"Restart {i}\n")
            file.write(f'Lap #: {filtered_values[i]+1}\n')
            file.write(f'Sort Lap: {filtered_values[i]+3}\n')
            out.to_csv(file, index=False)
            file.write("\n")

total = pd.concat(df_list)
lane_sum = total.groupby(['lane','restart_pos']).gain.mean().reset_index()
car_sum = total.groupby('vehicle_no').gain.mean().reset_index()
twentythreexi = total[total['vehicle_no'].isin(['23','45'])]
twentythreexi.columns = ['Vehicle No', 'Lane', 'Row', 'Post-Choose Position', 'Post-Sort Position', 'Gain/Loss']
twentythreexi['Lap'] = [i+1 for t in zip(filtered_values,filtered_values) for i in t]
twentythreexi.sort_values(['Lap', 'Vehicle No'])
twentythreexi.iloc[0:2,twentythreexi.columns.get_loc('Lap')] = 0 
z = []
for i in np.unique(twentythreexi.Lap):
    pre_choose = pd.read_sql(f'SELECT * FROM core.erdp_result WHERE race_id = {race_id} AND vehicle_no IN (23,45) AND lead_lap = {i-2} AND run_type = 3', mydb)
    pre_choose = pre_choose[['vehicle_no','position','lead_lap']]
    z.append(pre_choose)
pre_choose = pd.concat(z)
twentythreexi['Pre-Choose Position'] = list(twentythreexi['Post-Choose Position'][0:2]) + list(pre_choose.position)
twentythreexi = twentythreexi[['Vehicle No', 'Lane', 'Row',  'Pre-Choose Position', 'Post-Choose Position','Post-Sort Position', 'Gain/Loss', 'Lap']]

with pd.ExcelWriter(f'{race_id}_summary.xlsx') as writer:
    lane_sum.to_excel(writer, sheet_name='Restarts by Lane and Spot', index=False)
    car_sum.to_excel(writer, sheet_name='Restarts by Car', index=False)
    twentythreexi.to_excel(writer, sheet_name='23XI Restarts')


In [5]:
start = pd.read_sql('SELECT * FROM core.erdp_location_23kan2 WHERE run_type = 3 AND lap_fraction < 2', mydb_stg)
start2 = start[start['lap_fraction'] < 1.1]
start2 = start2[start2['lap_fraction'] > 1]
start2 = start2.sort_values('location_timestamp', ascending = True).groupby('vehicle_no').first().reset_index()
start2['pos'] = start2.last_loop_timestamp.rank()
start2 = start2.sort_values('pos').reset_index()
start2[['vehicle_no','location_timestamp','pos']]
pos = list(zip(start2.vehicle_position_x, start2.vehicle_position_y))
dist_white_line = list()
for j in pos:
    dist_white_line.append(calc_inside_distance(j, white_line)[0])
start2['dist_wl'] = dist_white_line
start2['lane'] = start2.dist_wl.apply(lambda x: 'inside' if x <= np.median(dist_white_line) else 'outside')
start2['restart_pos'] = start2.groupby('lane').pos.rank()
start2[['vehicle_no','restart_pos','dist_wl', 'lane', 'lap_fraction','pos','last_loop_name']].sort_values(['restart_pos','lane','pos'])

NameError: name 'mydb_stg' is not defined

In [ ]:
data = pd.read_sql(f'SELECT * FROM core.erdp_line_crossing WHERE race_id = {race_id} AND run_type = 3', mydb)
pi = data[data['loop_name'] == 'PI']
pi = pi[pi['lead_lap'] != 0]
pi = pi.sort_values(['lead_lap','vehicle_no'])
pi['ranked_pi'] = pi.groupby('lead_lap').position.rank(ascending=True).values
pi = pi[['vehicle_no','lead_lap','ranked_pi']]
po = data[data['loop_name'] == 'PO']
po = po[po['lead_lap'] != 0]
po = po.sort_values(['lead_lap','vehicle_no'])
po['ranked_po'] = po.groupby('lead_lap').position.rank(ascending = True).values
po.lead_lap = po.lead_lap - 1
po = po[['vehicle_no','lead_lap','ranked_po']]
pitroad = pd.merge(pi, po)